In [1]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('', 'TFIDF'))
sys.path.append(os.path.join('', 'util'))
from timer import Timer

In [ ]:
import TFIDF_numpy as tfidf

In [ ]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [18]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:20000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1
    
output = open('wordIndices.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000


In [ ]:
# Determine Term Frequencies
tf_vectors = []
for text in question_texts :
    vector = np.zeros(len(word_indices))
    for word in text :
        if word in word_indices :
            vector[word_indices[word]] += 1
    tf_vectors.append(vector)
tf_vectors = np.array(tf_vectors)

# Determine IDFS
num_docs_vector = np.zeros(len(word_indices))
N = float(len(questions))
for text in question_texts :
    temp_dict = {}
    for word in text :
        if word in word_indices :
            if word in temp_dict :
                continue
            else :
                temp_dict[word] = True
                num_docs_vector[word_indices[word]] += 1

idf_vector = np.log(N / num_docs_vector)                
tfidf_vectors = tf_vectors * idf_vector[None, :]
tfidf_norms = np.linalg.norm(tfidf_vectors, axis=1)

example = tfidf_vectors[0]
# results = tfidf_vectors.dot(example) / (np.linalg.norm(example) * np.ones(20000))
tfidf_vectors.dot(example) / (np.linalg.norm(example) * tfidf_norms)

In [ ]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.create_tfs()
tfidf.calculate_tfidfs()
tfidf.calculate_cossim(question_texts[0])

In [19]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts.pkl', 'rb'))

# Preprocess for cython code
# question_texts_c = np.array(list(word_indices.iteritems()))
tfidf_c.init_globals()
# tfidf.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.get_value_at_key("data.t")
# tfidf.create_tfs()
# tfidf.calculate_tfidfs()
# tfidf.calculate_cossim(question_texts[0])

for (key, value) in word_indices.iteritems() :
    if tfidf_c.get_value_at_key(key) != value :
        print key
#         print key.encode('utf-8', 'ignore')
    

Compiling with /usr/local/bin/clang-omp
1.4066696167e-05
0.0365869998932


In [ ]:
print word_indices

In [ ]:
dir(tfidf_c)

In [ ]:
with Timer() as t :
    a = list(word_indices.iteritems())
print t.interval
with Timer() as t :
    b = dict(a)
print t.interval

In [ ]:
import xxhash


In [ ]:
simhashes = []
for u in range(len(question_texts)):
    W = np.zeros(64)
    for i in range(len(question_texts[u])):
#         print (question_texts[u][i])
        try:
            wordhash = xxhash.xxh64(question_texts[u][i]).intdigest()
            counter = 63
            while wordhash > 0:
                bit = wordhash % 2
                if bit:
                    W[counter] += tfidf_vectors[u][i]
                else:
                    W[counter] -= tfidf_vectors[u][i]
                wordhash = wordhash >> 1
                counter -= 1
        except:
            pass
    simhash = 0
    for i in range(len(W)):
        if W[i] >= 0:
            simhash += 1
        if i < len(W)-1:
            simhash = simhash << 1
    simhashes.append(simhash)

In [ ]:
simhashes[10]




In [ ]:
for i in xrange(len(simhashes)):
    for j in xrange(i, len(simhashes)):
        if i != j:
            

In [ ]:
numpy_questions

In [ ]:
np.array(list(word_indices.iteritems()))

In [ ]:
a = np.array(list(word_indices.iteritems()))
a[:,0]

u'0xe2'